In [4]:
# librairies

import xarray as xr
import numpy as np
import pandas as pd
import glob
from datetime import datetime
import gc
import os
import sys

# Add the path to the custom library
#custom_library_path = os.path.abspath('/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/fabienVED')
#sys.path.append(custom_library_path)

#import data_process
from tqdm import tqdm
import matplotlib.pyplot as plt

# Loop

In [2]:
# paths
#upath = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/10m_u_component_of_wind/'
#vpath = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/10m_v_component_of_wind/'
ifg = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/instantaneous_10m_wind_gust/'

# Date specification
year = np.arange(1990,2021,1)
#target_month = 2
#target_day = 1

In [3]:
# Gather all the starting days of the 96 storms from the time series datasets


# Load the time series datasets

dates = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/storm_dates.csv')# pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/time_series_1h_EU/instantaneous_10m_wind_gust/instantaneous_10m_wind_gust_max.csv')
dates = dates['start_date']

# Extract the starting days of the 96 storms

storm_dates = []
for date in dates:
    storm_dates.append(datetime.strptime(date, '%Y-%m-%dT%H:%M:%S'))

# Extract each month and day of the storm starting days

storm_year = []
storm_month = []
storm_day = []
for date in storm_dates:
    storm_year.append(date.year)
    storm_month.append(date.month)
    storm_day.append(date.day)

# Combine the month and day of the storm starting days into a single list

storm_year_month_day = []
storm_month_day = []
for i in range(len(storm_month)):
    storm_year_month_day.append([storm_year[i],storm_month[i], storm_day[i]])
    storm_month_day.append([storm_month[i], storm_day[i]])

# keep the only the days that don't repeat themselves
storm_month_day = np.array(storm_month_day)
storm_month_day = np.unique(storm_month_day, axis=0)

storm_year_month_day = np.array(storm_year_month_day)
storm_year_month_day = np.unique(storm_year_month_day, axis=0)


In [ ]:
max_winds_europe = []
for target_month, target_day in tqdm(storm_month_day):
    for yearz in tqdm(year):
        # import data
        #u10path = upath+f'ERA5_{str(yearz)}-{str(target_month)}_10m_u_component_of_wind.nc'
        #v10path = vpath+f'ERA5_{str(yearz)}-{str(target_month)}_10m_v_component_of_wind.nc'
        i10fgpath = ifg+f'ERA5_{str(yearz)}-{str(target_month)}_instantaneous_10m_wind_gust.nc'
        
        # read files with xarray
        #u10 = xr.open_dataset(glob.glob(u10path)[0])
        #v10 = xr.open_dataset(glob.glob(v10path)[0])
        i10fg = xr.open_dataset(glob.glob(i10fgpath)[0])

        if target_month == 2 and target_day == 29:
            if yearz % 4 != 0:
                continue
            elif yearz % 100 == 0 and yearz % 400 != 0:
                continue

        # parse date
        first_true_index,last_true_index = data_process.parse_date_and_output_list(i10fg,target_month,target_day)

        # Convert longitude from 0-360 to -180 to 180
        #u10['longitude'] = ((u10['longitude'] + 180) % 360) - 180
        #v10['longitude'] = ((v10['longitude'] + 180) % 360) - 180
        i10fg['longitude'] = ((i10fg['longitude'] + 180) % 360) - 180
        # Sort the dataset along the new longitude axis
        #u10 = u10.sortby('longitude')
        #v10 = v10.sortby('longitude')
        i10fg = i10fg.sortby('longitude')

        # Get the Europe subregion
        #u10_europe = u10.sel(latitude=slice(70, 30), longitude=slice(-15, 40))
        #v10_europe = v10.sel(latitude=slice(70, 30), longitude=slice(-15, 40))
        i10fg_europe = i10fg.sel(latitude=slice(71, 30), longitude=slice(-15, 40))

        # parse date
        #u10_europe_date = u10_europe.isel(time=slice(first_true_index, last_true_index + 1))
        #v10_europe_date = v10_europe.isel(time=slice(first_true_index, last_true_index + 1))
        i10fg_europe_date = i10fg_europe.isel(time=slice(first_true_index, last_true_index + 1))

        # get wind speed
        #wspd = (u10_europe_date['10u']**2+v10_europe_date['10v']**2)**(0.5)
        #del u10,v10,u10_europe,v10_europe,u10_europe_date,v10_europe_date
        del i10fg, i10fg_europe
        gc.collect()

        # get maximum wind for 1 year
        #max_wind_europe = wspd.max(dim='time')
        max_wind_europe = i10fg_europe_date.max(dim='time')

        max_winds_europe.append(max_wind_europe)
    
    # Save the maximum wind speed for day target_day of month target_month
    locals()['max_winds_europe_'+str(target_month)+'_'+str(target_day)] = xr.concat(max_winds_europe, dim="time")#.mean('time')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [48:30<00:00, 45.47s/it]


In [5]:
#xr.concat(max_winds_europe, dim="time").mean('time').plot()
#test['i10fg'].plot()
for target_month, target_day in storm_month_day:
    ds = locals()['max_winds_europe_'+str(target_month)+'_'+str(target_day)]
    # Convert the dataset to a DataArray (if needed) and set CRS
    data_array = ds['i10fg']
    data_array = data_array.rio.write_crs("EPSG:4326")  # Assign CRS (WGS84)

    # Save as a raster file (GeoTIFF)
    output_path = f"/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/climatology/instantaneous_10m_wind_gust/{'max_winds_europe_'+str(target_month)+'_'+str(target_day)}.tif"
    data_array.rio.to_raster(output_path)
'''
#plt.title('Daily Max Wind Climatology: 01.02')
#plt.savefig('./climatology.png',dpi=400)'''

"\n#plt.title('Daily Max Wind Climatology: 01.02')\n#plt.savefig('./climatology.png',dpi=400)"